In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import numpy as np

In [ ]:
#read in the exisiting dataframe with listed U.S. companies
us_cos = pd.read_csv("us_stocks.csv")
us_cos

In [ ]:
#create new dataframe to hold the Sustainalytics scores
df_sustain = pd.DataFrame(index=['Sustainalytics'])
df_sustain

In [ ]:
#alter the exchanges to correspond to the Sustainalytics format
us_cos['exchange'].replace('NYSE','NYS', regex=True, inplace=True)
us_cos['exchange'].replace('NASDAQ','NAS', regex=True, inplace=True)
us_cos['exchange'].replace('AMEX','ASE', regex=True, inplace=True)
us_cos

In [ ]:
#create a new column that combines the ticker and the exchange
us_cos['exchange:ticker'] = us_cos[['exchange','ticker']].agg(':'.join, axis=1)

us_cos

In [ ]:
#launch automated Chrome browser and direct to website
chrome_driver = '/Users/tina/Desktop/Drivers/chromedriver'
driver = webdriver.Chrome(chrome_driver)

driver.get('https://www.sustainalytics.com/esg-rating/tesla-inc/1008215020')
time.sleep(2)

In [ ]:

#test = ['AAL','AABA','ABBV','ABC','ABCB']

counter = 0

for i in us_cos['exchange:ticker']:
    counter += 1
    page_source = driver.page_source
    soup = bs(page_source, 'lxml')
    element = driver.find_element(By.ID,"searchInput")
    element.clear()
    time.sleep(1.5)
    for character in i:
        element.send_keys(character)
        time.sleep(3) # pause in between characters
 
    
    try:
        co_dropdown = f"//span[@class='companyTicker' and text()='{i}']"
        elem = driver.find_element(By.XPATH, co_dropdown)
        parent = elem.find_element(By.XPATH,"../../../..")
        href = parent.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        time.sleep(1)
        driver.get(href)
        time.sleep(2)
        score = soup.find(class_ = 'col-6 risk-rating-score').find('span').get_text()
        df_sustain[i] = score
    except NoSuchElementException:
        pass
  
    time.sleep(1)  
    
    df_sustain.to_csv("sustainalytics_scores.csv", index=False)
    
    if counter % 100 == 0:
        time.sleep(10)
    
df_sustain  

In [ ]:
df_sustain

In [ ]:
driver.quit()
